# Advanced Querying Mongo

Importing libraries and setting up connection

In [2]:
from pymongo import MongoClient

In [3]:
import pymongo 

In [4]:
cliente = pymongo.MongoClient()

In [5]:
mydb = cliente["Companies"]

In [6]:
coleccion = mydb.detalles
coleccion

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Companies'), 'detalles')

In [7]:
import pandas as pd

In [8]:
pd.options.display.max_columns = None

In [9]:
df = pd.read_json("companies.json", lines=True)

In [44]:
df.head(3)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,founded_year,founded_month,founded_day,deadpooled_year,tag_list,alias_list,email_address,phone_number,description,created_at,updated_at,overview,image,products,relationships,competitions,providerships,total_money_raised,funding_rounds,investments,acquisition,acquisitions,offices,milestones,video_embeds,screenshots,external_links,partners,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,{'$oid': '52cdef7c4bab8bd675297d8a'},Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,2005.0,10.0,17.0,1.0,"wiki, seattle, elowitz, media-industry, media-...",,info@wetpaint.com,206.859.6300,Technology Platform Company,{'$date': 1180075887000},2013-12-08 07:15:44+00:00,<p>Wetpaint is a technology platform company t...,"{'available_sizes': [[[150, 75], 'assets/image...","[{'name': 'Wikison Wetpaint', 'permalink': 'we...","[{'is_past': False, 'title': 'Co-Founder and V...","[{'competitor': {'name': 'Wikia', 'permalink':...",[],$39.8M,"[{'id': 888, 'round_code': 'a', 'source_url': ...",[],"{'price_amount': 30000000, 'price_currency_cod...",[],"[{'description': '', 'address1': '710 - 2nd Av...","[{'id': 5869, 'description': 'Wetpaint named i...",[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],NaN,NaN,NaN,NaN
1,{'$oid': '52cdef7c4bab8bd675297d8b'},AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,1996.0,NaN,NaN,2.0,,Zoho ManageEngine,pr@adventnet.com,925-924-9500,Server Management Software,{'$date': 1180121062000},2012-10-31 18:26:09+00:00,"<p>AdventNet is now <a href=""/company/zoho-man...","{'available_sizes': [[[150, 55], 'assets/image...",[],"[{'is_past': True, 'title': 'CEO and Co-Founde...",[],"[{'title': 'DHFH', 'is_past': True, 'provider'...",$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...",[],[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN
2,{'$oid': '52cdef7c4bab8bd675297d8c'},Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,2005.0,9.0,15.0,3.0,"zoho, officesuite, spreadsheet, writer, projec...",,info@zohocorp.com,1-888-204-3539,Online Business Apps Suite,Fri May 25 19:30:28 UTC 2007,2013-10-30 00:07:05+00:00,"<p>Zoho offers a suite of Business, Collaborat...","{'available_sizes': [[[150, 55], 'assets/image...","[{'name': 'Zoho Office Suite', 'permalink': 'z...","[{'is_past': False, 'title': 'CEO and Founder'...","[{'competitor': {'name': 'Empressr', 'permalin...",[],$0,[],[],None,[],"[{'description': 'Headquarters', 'address1': '...","[{'id': 388, 'description': 'Zoho Reaches 2 Mi...","[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],NaN,NaN,NaN,NaN


In [11]:
df2=df.drop(["_id"],axis=1) """esque con el id no me dejaba subirlo"""

In [12]:
coleccion.insert_many(df2.to_dict("records"))

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [18]:
df_babelgum=pd.DataFrame(coleccion.find({"name":"Babelgum"}))

In [19]:
df_babelgum.name.unique()

array(['Babelgum'], dtype=object)

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [20]:
df_emp = pd.DataFrame(coleccion.find({'number_of_employees': {"$gt": 5000}}).limit(20))

In [27]:
df_emp.number_of_employees.unique()

array([  5299.,  15000.,  63000.,  13600.,  28000.,  86300.,   5080.,
         7173., 180500., 300000.,  25000.,   8000.,   8657.,  33350.,
       125000.,  10000.,  90000.,  80000.,  51000.])

In [28]:
df_emp.shape

(20, 42)

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [182]:
cond1={'founded_year': {"$gte": 2000}}
cond2={'founded_year': {"$lte": 2005}}

In [183]:
df_years = pd.DataFrame(coleccion.find({'$and': [cond1, cond2]}))

In [185]:
df_years=df_years[["name","founded_year"]]
df_years

,name,founded_year
0,Wetpaint,2005.0
1,Zoho,2005.0
2,Digg,2004.0
3,Facebook,2004.0
4,Omnidrive,2005.0
...,...,...
7463,EnterSys Group,2000.0
7464,Axon Solutions,2004.0
7465,Intergy,2003.0
7466,AfterLogic,2002.0


In [41]:
df_years.founded_year.unique()

array([2005., 2004., 2002., 2003., 2000., 2001.])

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [50]:
cond1={'total_money_raised': {"$gt": 100000000}}"""no encunetro el valuation amount"""
cond2={'founded_year': {"$lt": 2010}}

In [47]:
df_Val = pd.DataFrame(coleccion.find({'$and': [cond1, cond2]}))
df_Val

""


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [51]:
cond1={'number_of_employees': {"$lt": 1000}}
cond2={'founded_year': {"$lt": 2005}}

In [60]:
df_ey = pd.DataFrame(coleccion.find({'$and': [cond1, cond2]}).sort("number_of_employees", 1).limit(10))

### 6. All the companies that don't include the `partners` field.

In [61]:
list(coleccion.find({'partners': {'$exists': False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [76]:
df_nul=pd.DataFrame(coleccion.find({"category_code": {"$type": "null"}}, {"_id":0, "name":1}))

In [77]:
df_nul["name"].unique()

array(['Collective', 'Snimmer', 'KoolIM', ..., 'cruisecritic', 'Coloroot',
       'Geekdive'], dtype=object)

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [74]:
# Your Code
cond1={'number_of_employees': {"$gte": 100}}
cond2={'number_of_employees': {"$lt": 1000}} """sin dataframe"""

In [85]:
list(coleccion.find({'$and': [cond1, cond2]},{"name":1,"number_of_employees":1,"_id":0})[2:5])

[{'name': 'OpenX', 'number_of_employees': 305.0},
 {'name': 'LifeLock', 'number_of_employees': 644.0},
 {'name': 'Jajah', 'number_of_employees': 110.0}]

### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
list(coleccion.find().sort("ipo", 1))[0] """nose que pasa con ipo(tengo Nans) pero se haria asi"""

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [103]:
df_10=pd.DataFrame(coleccion.find().sort("number_of_employees", -1).limit(10))

In [104]:
df_10["name"].unique()

array(['Siemens', 'IBM', 'Toyota', 'PayPal',
       'Nippon Telegraph and Telephone Corporation'], dtype=object)

In [105]:
df_10.shape

(10, 42)

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [108]:
coleccion.distinct( "founded_month" ) #2nd semester sup a 6

[nan, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0]

In [111]:
df_sem=pd.DataFrame(coleccion.find({'founded_month': {"$gte": 6}}).limit(1000))

In [112]:
df_sem["founded_month"].unique()

array([10.,  9., 11.,  6., 12.,  8.,  7.])

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
cond1={'founded_years': {"$lt": 2000}}
cond2={'acquisitions': {"$gt": 1000000}} """"acquisitions" me sale como dict"""

In [ ]:
list(coleccion.find({'$and': [cond1, cond2]},{"name":1,"_id":0}).limit(1000000)) """sería así"""

In [124]:
df_adq=pd.DataFrame(coleccion.find({'founded_year': {"$lt": 2000}}).limit(1000))

In [127]:
df_adq["acquisitions"] """para las preguntas con "acquisitions" las dejo sin runear"""

0                                                     []
1                                                     []
2      [{'price_amount': 20000000, 'price_currency_co...
3      [{'price_amount': 1500000000, 'price_currency_...
4      [{'price_amount': 280000000, 'price_currency_c...
                             ...                        
995                                                   []
996                                                   []
997                                                   []
998    [{'price_amount': None, 'price_currency_code':...
999                                                   []
Name: acquisitions, Length: 1000, dtype: object

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [ ]:
list(coleccion.find({'founded_year': {"$gt": 2010}},{"_id":0, "name":1, "acquisitions":1}).sort("acquisitions", 1).limit(1000))

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [167]:
list(coleccion.find({'founded_year': {"$gt": 0}},{"_id":0, "name":1, "founded_year":1}).sort('founded_year',1))[0]

{'name': 'Alstrasoft', 'founded_year': 1800.0}

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
list(coleccion.find({"founded_day": {"$lte": 7}}).sort("acquisitions",1).limit(10) 

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [174]:
cond1={"category_code":"web"}
cond2={"number_of_employees":{"$gt":4000}}

In [178]:
list(coleccion.find({'$and': [cond1, cond2]},{"name":1,"number_of_employees":1,"_id":0}).sort("number_of_employees",1)[0:3])


[{'name': 'Expedia', 'number_of_employees': 4400.0},
 {'name': 'Expedia', 'number_of_employees': 4400.0},
 {'name': 'AOL', 'number_of_employees': 8000.0}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
cond1={"acquisitions":{"$gt":4000}}
cond2={"currency":"EUR"} """la divisa tampoco me sale"""

In [ ]:
list(coleccion.find({'$and': [cond1, cond2]},{"name":1,"_id":0})[0]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [180]:
list(coleccion.find({'founded_month': {"$lte": 3}},{"name":1,"_id":0}).limit(10))[0] """pongo solo el nombre xq "acquisitions" no me funciona"""

{'name': 'Facebook'}

In [ ]:
list(coleccion.find({'founded_month': {"$lte": 3}},{"name":1,"acquisitions":0,"_id":0}).limit(10))[0] #sería así

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code